In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from io import BytesIO

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import matplotlib.pyplot as plt

from data_collector import Omie, MarginalPriceParams, OfferCurvesParams

In [ ]:
filename = "curva_pbc"
date = "20211001"

date = pd.to_datetime(date).strftime("%Y%m%d")
filename_date = f"{filename}_{date}.1"

cols = [
    "hour", "date", "country", "unit", "offer_type", "energy", "price", "status"
]

In [ ]:
content = Omie._download_content(family_file=filename, filename=filename_date)

In [ ]:
filebytes = BytesIO(content)

In [ ]:
df = pd.read_csv(filebytes, delimiter=";", index_col=False, skiprows=3, names=cols, encoding="latin-1")
df.drop(["unit"], axis=1, inplace=True)
df.dropna(inplace=True)

df["price"] = df["price"].str.replace(".", "").str.replace(",", ".").astype(float)
df["energy"] = df["energy"].str.replace(".", "").str.replace(",", ".").astype(float)

In [ ]:
df.dropna(axis=0, how="all")

In [ ]:
df_1 = df[df["hour"] == 1]

In [ ]:
df_offer = df_1[(df_1["offer_type"] == "V") & (df_1["status"] == "O")].sort_values("price")
df_offer["agg_energy"] = df_offer["energy"].cumsum() 

In [ ]:
df_demand = df_1[(df_1["offer_type"] == "C") & (df_1["status"] == "O")].sort_values("price", ascending=False)
df_demand["agg_energy"] = df_demand["energy"].cumsum()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
df_offer.plot(x="agg_energy", y="price", ax=ax, label="ask")
df_demand.plot(x="agg_energy", y="price", ax=ax, label="bid")
ax.set_xlim(11000, 24000)
ax.set_ylim(-10, 300)

In [ ]:
df_units = pd.read_csv("curva_pbc_uof_20211001.1", delimiter=";", index_col=False, skiprows=3, encoding="latin-1",
                      names=cols)
#df_units.dropna(inplace=True)

df_units["price"] = df_units["price"].str.replace(".", "").str.replace(",", ".").astype(float)
df_units["energy"] = df_units["energy"].str.replace(".", "").str.replace(",", ".").astype(float)

In [ ]:
df_units

In [ ]:
df_units_offer = df_units[
    (df_units["hour"] == 1) & (df_units["offer_type"] == "V") & (df_units["status"] == "O")
].sort_values("price")

In [ ]:
df_offer

In [ ]:
df_units_offer.head(50)

In [ ]:
filename = "curva_pbc_uof"
year = 2016
month = 1
year_month = f"{year}{month:02d}"

date_file_pattern = f"{filename}_{year_month}"
filename_zip = date_file_pattern + ".zip"

In [ ]:
content = Omie._download_content(family_file=filename, filename=filename_zip)

In [ ]:
type(content)

In [ ]:
unzip_file = Omie._decompress_zip(zip_content=content)

In [ ]:
d = pd.to_datetime(f'{year}-{month}')
dates = pd.date_range(start=d.strftime("%Y-%m-%d"), periods=d.daysinmonth, freq="D")  

df_units_list = []
for d in dates:
    file_pattern = Omie.date_file_pattern.format(filename=filename, date_str=d.strftime("%Y%m%d"))
    file_list = [s for s in unzip_file.namelist() if file_pattern in s]
    
    if len(file_list) > 1:
        print(f"There are multiple files for {file_pattern}: {len(file_list)}")
    
    file = file_list[-1]

    filebytes = unzip_file.open(name=file, mode="r")
    
    df = pd.read_csv(filepath_or_buffer=filebytes, 
                     delimiter=";", index_col=False, skiprows=3, encoding="latin-1",
                     names=cols)
    df = Omie._parse_floats(df=df, col_name="price")
    df = Omie._parse_floats(df=df, col_name="energy")
    df.dropna(inplace=True)
    
    df_units_list.append(df)

In [ ]:
df_curves_month = pd.concat(df_units_list, ignore_index=True)

In [ ]:
df_curves_month

In [ ]:
pd.to_datetime(df_curves_month["date"], format="%d/%m/%Y")

In [ ]:
cc_unit_offers = df_curves_month[(df_curves_month["unit"] == "SBO3") & 
                (df_curves_month["status"] == "O")].sort_values(["date", "hour"])

In [ ]:
cc_unit_offers["total_enegy"] = cc_unit_offers.groupby(['date', "hour"])['energy'].transform('sum')
cc_unit_offers["weighted_price"] = (cc_unit_offers["price"] * cc_unit_offers["energy"]) /\
                                    cc_unit_offers["total_enegy"]

In [ ]:
cc_unit_offers

In [ ]:
cc_unit_offers.groupby(["date", "hour"])["weighted_price"].sum().plot(figsize=(20, 7))

In [ ]:
cc_unit_offers.groupby(["date", "hour"])["weighted_price"].sum().head(50)

In [ ]:
cc_unit_offers.head(50)

In [3]:
df = Omie.download_date_file(omie_parameter=MarginalPriceParams, date="2021-11-01")

In [4]:
df_2 = Omie.download_date_file(omie_parameter=OfferCurvesParams, date="2021-11-01")

In [5]:
df

,year,month,day,hour,portugal,spain,date
0,2021,11,1,1,78.43,78.43,2021-11-01
1,2021,11,1,2,55.06,55.06,2021-11-01
2,2021,11,1,3,43.92,43.92,2021-11-01
3,2021,11,1,4,22.05,22.05,2021-11-01
4,2021,11,1,5,15.00,15.00,2021-11-01
5,2021,11,1,6,41.09,41.09,2021-11-01
6,2021,11,1,7,58.29,58.29,2021-11-01
7,2021,11,1,8,77.54,77.54,2021-11-01
8,2021,11,1,9,88.81,88.81,2021-11-01
9,2021,11,1,10,85.42,85.42,2021-11-01


In [6]:
df_2

,hour,date,country,unit,offer_type,energy,price,status
0,1,2021-11-01,MI,NaN,C,238.3,505.00,O
1,1,2021-11-01,MI,NaN,C,73.2,501.00,O
2,1,2021-11-01,MI,NaN,C,39.2,500.00,O
3,1,2021-11-01,MI,NaN,C,17.6,500.00,O
4,1,2021-11-01,MI,NaN,C,1.0,500.00,O
...,...,...,...,...,...,...,...,...
84031,24,2021-11-01,MI,NaN,V,80.0,119.89,C
84032,24,2021-11-01,MI,NaN,V,28.9,120.00,C
84033,24,2021-11-01,MI,NaN,V,119.0,120.00,C
84034,24,2021-11-01,MI,NaN,V,119.0,126.00,C
